<a href="https://colab.research.google.com/github/bdav22/CSC370/blob/master/HW_X_Spark_RDD_Word_Count_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW_X: Word Count
    - 10 points
    - due by the midnight on April 26.
    - submit your notebook to the Blackboard.


---


## Requirments: You need to use PySpark RDD to do the HW. 

    0. [1 points]  Spark Session Creation
        - use local mode
        - use Spark Context to create rdd 
    
    1. [1 point] Upload the "Shakespeare.txt" to Colab file system
        you need to download the book from the BB to your computer first
        - check how many lines
        - print out the first 10 records
    
    2. [6 points] Word count
        - use MapReduce
        - sort by count

    3. [1 point] Save a csv file to disk
        - convert to Pandas DataFrame
        - save to colab

    4. [1 point] Submit 2 files to the BB
        a. your notebook without any errors
        b. the csv file which stors the word count (descending order by count)



---
RDD Ref: https://sparkbyexamples.com/pyspark-rdd/




---


## 0. Spark Session Creation


---



In [2]:
!pip install pyspark  # Install PySpark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=ac0ee961d0931d8c4a355fc7819b6d2b0a38f5c029018b4b9c95fa35ede87100
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [5]:
# Import SparkSession first
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf


# Create Spark Session first using local master and word_count as the app name.
spark = SparkSession.builder.appName("word_count").getOrCreate()

# display spark session 
spark

# word_count.stop()



---


##  1. Create An RDD using Spark Context by reading in file
    

---



In [7]:
# (1) upload  "shakespears.txt" to the Colab default location
# on the left panel, click the upload icon 


# (2) read in local file, generate a RDD, and call it rdd
wordFile = "shakespeare.txt"
rdd =  spark.read.text(wordFile).rdd


# display the type of rdd
print(type(rdd))



<class 'pyspark.rdd.RDD'>


In [8]:
# count how many lines: total 4155 lines
line_count = rdd.count()
print("Line count: ", line_count)



# take the first 10 records 
first10 = rdd.take(10)
for record in first10:
  print(record)



Line count:  4155
Row(value="All's Well That Ends Well")
Row(value="Shakespeare homepage | All's Well That Ends Well | Entire play")
Row(value='ACT I')
Row(value='')
Row(value="SCENE I. Rousillon. The COUNT's palace.")
Row(value='')
Row(value='Enter BERTRAM, the COUNTESS of Rousillon, HELENA, and LAFEU, all in black')
Row(value='COUNTESS')
Row(value='In delivering my son from me, I bury a second husband.')
Row(value='BERTRAM')




---
## 2. Word Count using MapReduce
---



In [9]:
# split each line to single word using split() method, name the new rdd as word_list
word_list = rdd.map(lambda line: line.value.split())


# print the number of words in word_list: 24251
words = word_list.flatMap(lambda word_list: word_list)
word_count = words.count()
print("Word count", word_count)


# display the first 10 words in word_list
first10Words = word_list.flatMap(lambda word_list: word_list).take(10)
print("First 10 words: ", first10Words)



Word count 24251
First 10 words:  ["All's", 'Well', 'That', 'Ends', 'Well', 'Shakespeare', 'homepage', '|', "All's", 'Well']


In [10]:
# map each word to low case using lower() method, name the new RDD as low_list
low_list = word_list.flatMap(lambda word_list: word_list).map(lambda word: word.lower())




# display the first 5
first5Low = low_list.take(5)
first5Low



["all's", 'well', 'that', 'ends', 'well']

In [11]:
# remove all punctions in the low_list using strip(puntuation) method, name it as low_word
from string import punctuation
import string

low_word = low_list.map(lambda word: word.strip(string.punctuation))

# count how many words in the RDD low_word: 24251
low_count = low_word.count()
print("Num of words: ", low_count)
# the first 10:
print(low_word.take(10))

Num of words:  24251
["all's", 'well', 'that', 'ends', 'well', 'shakespeare', 'homepage', '', "all's", 'well']


In [14]:
# Notice that there are empty string in the RDD low_word, remove all missing values using filter 
word_clear = low_word.filter(lambda x: x != '')


# count how many words in word_clear: 24246
clear_count = word_clear.count()
print("Num of words: ", clear_count)



Num of words:  24246


In [18]:
# Word Count: map each word to 1, reduce all the words based on Key,  name it as unique_count
unique_count = word_clear.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)



# count how many unique word in unique_count: 3562
print("Unique words: ", unique_count.count())


# display the first 5 
print("First 5: ", unique_count.take(5))



Unique words:  3562
First 5:  [("all's", 5), ('well', 84), ('that', 325), ('ends', 5), ('shakespeare', 1)]


In [ ]:
# sort the RDD unique_count by count, name it sorted_word




#print most frequent 10 words





[(731, 'the'), (708, 'i'), (620, 'and'), (518, 'to'), (484, 'you'), (459, 'of'), (453, 'a'), (378, 'my'), (325, 'that'), (300, 'in')]


In [ ]:
# using map to reverse the RDD sorted_words order (count, word) to (word, count), name it as final_word



# display the first 10





[('the', 731), ('i', 708), ('and', 620), ('to', 518), ('you', 484), ('of', 459), ('a', 453), ('my', 378), ('that', 325), ('in', 300)]




---


### 3. Save a csv file to disk
        - convert to Pandas DataFrame
        - save the dataframe to the colab using the name "word_count.csv"
        - download the csv file to your computer

---



In [ ]:
# RDD to Spark DF


# to Pandas DF and save it to colab






---
### 4. Submit 2 files to the BB

    a. this notebook (rerun your code to make sure there is no error) and
    b. "word_count.csv" file 

---




In [ ]:
# stop your spark session
spark.stop()